# Making Your Model Learn Addition!
## Task 1: Introduction

Given the string "54+7", the model should return a prediction: "61".

In [2]:
import numpy as np
from tensorflow import keras
from keras import models
from keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Dense, Dropout, SimpleRNN, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback

from termcolor import colored

## Task 2: Generate Data
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [3]:
all_chars = '0123456789+'

In [4]:
num_features = len(all_chars)

char_to_index = dict((c, i) for i, c in enumerate(all_chars))
index_to_char = dict((i, c) for i, c in enumerate(all_chars))

print('Number of features:', num_features)

Number of features: 11


In [5]:
def generate_data():
    first_num = np.random.randint(low=0,high=100)
    second_num = np.random.randint(low=0,high=100)
    example = str(first_num) + '+' + str(second_num)
    label = str(first_num+second_num)
    return example, label

generate_data()

('67+22', '89')

## Task 3: Create the Model
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___
Consider these two reviews:

Review 1: This movie is not terrible at all.

Review 2: This movie is pretty decent.

In [6]:
hidden_units = 128
max_time_steps = 5

model = Sequential([
    SimpleRNN(hidden_units, input_shape=(None, num_features)),
    RepeatVector(max_time_steps),
    SimpleRNN(hidden_units, return_sequences=True),
    TimeDistributed(Dense(num_features, activation='softmax'))
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 128)               17920     
                                                                 
 repeat_vector (RepeatVector  (None, 5, 128)           0         
 )                                                               
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 5, 128)            32896     
                                                                 
 time_distributed (TimeDistr  (None, 5, 11)            1419      
 ibuted)                                                         
                                                                 
Total params: 52,235
Trainable params: 52,235
Non-trainable params: 0
_________________________________________________________________


## Task 4: Vectorize and De-Vectorize Data
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [7]:
def vectorize_example(example, label):
    
    x = np.zeros((max_time_steps, num_features))
    y = np.zeros((max_time_steps, num_features))
    
    diff_x = max_time_steps - len(example)
    diff_y = max_time_steps - len(label)
    
    for i, c in enumerate(example):
        x[diff_x+i, char_to_index[c]] = 1
    for i in range(diff_x):
        x[i, char_to_index['0']] = 1
    for i, c in enumerate(label):
        y[diff_y+i, char_to_index[c]] = 1
    for i in range(diff_y):
        y[i, char_to_index['0']] = 1
        
    return x, y

e, l = generate_data()
print('Text Example and Label:', e, l)
x, y = vectorize_example(e, l)
print('Vectorized Example and Label Shapes:', x.shape, y.shape)

Text Example and Label: 45+5 50
Vectorized Example and Label Shapes: (5, 11) (5, 11)


In [8]:
def devectorize_example(example):
    result = [index_to_char[np.argmax(vec)] for i, vec in enumerate(example)]
    return ''.join(result)

devectorize_example(x)

'045+5'

In [9]:
devectorize_example(y)

'00050'

## Task 5: Create Dataset
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [10]:
def create_dataset(num_examples=2000):

    x_train = np.zeros((num_examples, max_time_steps, num_features))
    y_train = np.zeros((num_examples, max_time_steps, num_features))

    for i in range(num_examples):
        e, l = generate_data()
        x, y = vectorize_example(e, l)
        x_train[i] = x
        y_train[i] = y
    
    return x_train, y_train

x_train, y_train = create_dataset()
print(x_train.shape, y_train.shape)

(2000, 5, 11) (2000, 5, 11)


In [11]:
devectorize_example(x_train[0])

'25+32'

In [12]:
devectorize_example(y_train[0])

'00057'

## Task 6: Training the Model
___
Note: If you are starting the notebook from this task, you can run cells from all previous tasks in the kernel by going to the top menu and then selecting Kernel > Restart and Run All
___

In [15]:
l_cab = LambdaCallback(
    on_epoch_end = lambda e, l: print('{:.2f}'.format(l['val_acc']), end=' _ ')
)

es_cb = EarlyStopping(monitor='val_loss', patience=10)

model.fit(
    x_train,
    y_train,
    epochs = 500,
    batch_size = 256,
    validation_split = .2,
    verbose = False,
#    callbacks=[l_cab, es_cb]
)

In [18]:
x_test, y_test = create_dataset(50)

preds = model.predict(x_test)

for i, pred in enumerate(preds):
    y = devectorize_example(y_test[i])
    y_hat = devectorize_example(pred)
    col = 'green'
    if y != y_hat:
        col = 'red'
        
    out = 'Input: ' + devectorize_example(x_test[i]) + ' Out: ' + y  + ' Pred: ' + y_hat
    print(colored(out, col))
    

2/2 [==============================] - 0s 10ms/step
Input: 96+51 Out: 00147 Pred: 00157
Input: 040+8 Out: 00048 Pred: 00048
Input: 39+89 Out: 00128 Pred: 00128
Input: 54+72 Out: 00126 Pred: 00126
Input: 36+91 Out: 00127 Pred: 00127
Input: 00+93 Out: 00093 Pred: 00094
Input: 68+47 Out: 00115 Pred: 00115
Input: 70+82 Out: 00152 Pred: 00152
Input: 20+21 Out: 00041 Pred: 00041
Input: 77+19 Out: 00096 Pred: 00096
Input: 90+20 Out: 00110 Pred: 00110
Input: 056+2 Out: 00058 Pred: 00058
Input: 90+68 Out: 00158 Pred: 00158
Input: 94+81 Out: 00175 Pred: 00175
Input: 96+99 Out: 00195 Pred: 00195
Input: 30+70 Out: 00100 Pred: 00100
Input: 06+81 Out: 00087 Pred: 00087
Input: 42+43 Out: 00085 Pred: 00085
Input: 63+33 Out: 00096 Pred: 00096
Input: 94+88 Out: 00182 Pred: 00182
Input: 76+78 Out: 00154 Pred: 00154
Input: 91+66 Out: 00157 Pred: 00157
Input: 99+81 Out: 00180 Pred: 00180
Input: 85+54 Out: 00139 Pred: 00139
Input: 023+2 Out: 00025 Pred: 00024
Input: 061+1 Out: 00062 Pred: 00065
Input: 24+87